In [135]:
from random import random, choice, randint
from collections import namedtuple
from pprint import pprint, pformat
import random
import numpy as np
from copy import copy, deepcopy
import logging
from dataclasses import dataclass


In [136]:
#  code same as teacher
Nimply = namedtuple("Nimply", "row, num_objects")


In [137]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


In [138]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [139]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [140]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


In [141]:
#change start
#asked paul-raphael about his code and see that I was in the wrong direction, so I took his code and I tried to modifie this. The concept is that this ea is used to create a list of weigth which is used to choose a strategy to apply each turn 

POPULATION_SIZE = 40
OFFSPRING_SIZE = 15
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .80
 #if there are more strategie rulse size increase 
list_strategies= [pure_random, gabriele, optimal]
#the number of strategie
RULES_SIZE= len(list_strategies)
size_game=4


num_generations=200


In [142]:


@dataclass
class Individual:
    fitness: float
    genotype: []

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

#mutate def from paul raphael 
def mutate(individual) ->Individual :     #changes the genotype of the individual by applying a small random change to one of the values
    genotype= individual.genotype
    # Choose an index to change
    index_to_change = random.randint(0, len(genotype) - 1)

    # Generate a random value for the change
    change = random.uniform(-0.05, 0.05)  # You can adjust the range as needed

    # Apply the change to the chosen index
    genotype[index_to_change] += change

    # Ensure the modified value is between 0 and 1
    genotype[index_to_change] = max(0, min(1, genotype[index_to_change]))

    # Adjust the other values to maintain the sum equal to 1
    total = sum(genotype)
    genotype = [value / total for value in genotype]

    individual.genotype= genotype
    return individual



In [143]:
def play(first,genotype , strategy_player1):
    # Choose the best PossibleMove in the population and play Nim against the pure random strategy
    
    nim = Nim(size_game)
    player = first
    while nim:
        #here the strat is randomly choose from the weigth of genotype
        chosen_strat = random.choices(list_strategies, genotype)[0]
        strategy = ( chosen_strat, strategy_player1)
        ply = strategy[player](nim)
        nim.nimming(ply)
        player = 1 - player
    
    return player


In [144]:
def fitness(genotype):#returns the rate of winning the game after playing it 'nb_game' times with one same genotype mbut against differents types of opponent's strategies
    win=0
    first=0
    strategies_opponent= list_strategies
    nb_game=30
    for i in range (nb_game):
        opponent= strategies_opponent[i%RULES_SIZE] # A verifier les effets de bords
        val= play(first, genotype, opponent)
        first=1-first
        if val == 0:
            win+=1
    
    win_rate= win/nb_game
    return win_rate

In [202]:
def evolved_startegy():
    population = [
        Individual(
            fitness=None,
            genotype = np.random.dirichlet(np.ones(RULES_SIZE), size = 1)[0]              
        )
        for _ in range(POPULATION_SIZE)
    ]

    for i in population:
        i.fitness = fitness(i.genotype)  # Use the fitness function on the nim_move

    for generation in range(num_generations):
        offspring = list()
        for counter in range(OFFSPRING_SIZE):
            p1 = select_parent(population)
            if random.random() < MUTATION_PROBABILITY:                                  
                p1 = mutate(p1)
                            
            offspring.append(p1)

        for i in offspring:
            i.fitness = fitness(i.genotype) 
            
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse = True)           
        population = population[:POPULATION_SIZE]

    # Choose the best individual in the population and return its genotype
    print('best individual')
    print(population[0].fitness)
    print(population[0].genotype)
    return population[0].genotype

chosen_genotype=evolved_startegy()


best individual
0.4
[0.2511338534887735, 0.34388357221788224, 0.4049825742933443]


## Oversimplified match

In [204]:

        
logging.getLogger().setLevel(logging.INFO)


nim = Nim(size_game)
logging.info(f"init : {nim}")
logging.info(f"current_geno : {chosen_genotype}")
player = 0
while nim:
    #here the strat is randomly choose from the weigth of genotype
    strategy = (random.choices(list_strategies, chosen_genotype)[0], pure_random)
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7>
INFO:root:current_geno : [0.2511338534887735, 0.34388357221788224, 0.4049825742933443]
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=3)
INFO:root:status: <1 3 5 4>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 5 4>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 3 4 4>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=3)
INFO:root:status: <0 3 1 4>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=2)
INFO:root:status: <0 1 1 4>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 1 0 4>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 0 0 4>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 0 0 3>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=2)
INFO:root:status: <0 0 0 1>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 0 0 0>
INFO:root:status